# Introduction   


We will experiment now with the Mistral model.


## Model specification  

The model details are:

* **Model**: Mistral
* **Variation**: 7b-v0.1-hf (7b: 7B dimm. hf: HuggingFace build)
* **Version**: V1
* **Framework**: PyTorch

# Install and import packages  

In [1]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [2]:
!pip install -q -U langchain

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,>=1.3, but 

In [3]:
import torch
from time import time
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain import PromptTemplate

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Define model  

In [4]:
model_id = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'
time_1 = time()
tokenizer = AutoTokenizer.from_pretrained(model_id)
model_name = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )
print(f"Tokenizer & pipeline: {round(time() - time_1)} sec.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Tokenizer & pipeline: 178 sec.


# Test model  

Let's test the model.

In [5]:
time_1 = time()
query_pipeline = pipeline(
        "text-generation",
        model=model_name,
        tokenizer=tokenizer,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        torch_dtype=torch.float16,
        device_map="auto",
        max_length=256,)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 0.0 sec.



Let's define a function to test the query pipeline.


In [6]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

Let's try a question about touristic attractions in France.

In [7]:
test_model(tokenizer,
           query_pipeline,
           "Please let me know which cities are mostly visited in France.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 15.528 sec.
Result: Please let me know which cities are mostly visited in France.

Comment: I'm not sure if you're asking about the most visited cities in France or the most visited cities in the world.

Comment: @JonathanReez I'm asking about the most visited cities in France.

Comment: @JonathanReez I'm asking about the most visited cities in France.

Comment: @JonathanReez I'm asking about the most visited cities in France.

Comment: @JonathanReez I'm asking about the most visited cities in France.

Comment: @JonathanReez I'm asking about the most visited cities in France.

Comment: @JonathanReez I'm asking about the most visited cities in France.

Comment: @JonathanReez I'm asking about the most visited cities in France.

Comment:


Let's adjust the prompt, since we ar not really happy with this answer.

In [8]:
test_model(tokenizer,
           query_pipeline,
           "What are the three most visited cities in France?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 1.623 sec.
Result: What are the three most visited cities in France?

The three most visited cities in France are Paris, Marseille, and Lyon.


In [9]:
test_model(tokenizer,
           query_pipeline,
           "What are the three most visited tourist attractions in Paris?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Test inference: 2.374 sec.
Result: What are the three most visited tourist attractions in Paris?

The three most visited tourist attractions in Paris are the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.


It looks like how the prompt is created is really important.

# Define and execute the sequential chain  



In [10]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

In [11]:
llm("What are the three most visited cities in France?")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'What are the three most visited cities in France?\n\nThe three most visited cities in France are Paris, Marseille, and Lyon.'

In [12]:
def sequential_chain(country, llm):
    """
    Args:
        country: country selected
    Returns:
        None
    """
    time_1 = time()
    template = "What is the name of the most popular city in {country} for tourists?"

    #  first task in chain
    first_prompt = PromptTemplate(
        input_variables=["country"],
        template=template)

    city = LLMChain(llm = llm, prompt = first_prompt)

    # second step in chain
    second_prompt = PromptTemplate(
        input_variables=["city"],
        template="What are the top three things to do in this {city} for tourists?")

    chain_two = LLMChain(llm=llm, prompt=second_prompt)

    # combine the two steps and run the chain sequence
    overall_chain = SimpleSequentialChain(chains=[city, chain_two], verbose=True)
    overall_chain.run(country)
    time_2 = time()
    print(f"Run sequential chain: {round(time_2-time_1, 3)} sec.")

In [13]:
final_answer = sequential_chain("France", llm)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new SimpleSequentialChain chain...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the name of the most popular city in France for tourists?

Paris is the most popular city in France for tourists. Known as the City of Light, Paris is famous for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. The city is also known for its romantic atmosphere, world-class cuisine, and fashion.
What are the top three things to do in this What is the name of the most popular city in France for tourists?

Paris is the most popular city in France for tourists. Known as the City of Light, Paris is famous for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. The city is also known for its romantic atmosphere, world-class cuisine, and fashion. for tourists?

1. Visit the Eiffel Tower: The Eiffel Tower is one of the most famous landmarks in the world and a must-see attraction for tourists in Paris. Visitors can take an elevator to the top of the tower for stunning views of the city.
2. Explore the Lo

In [14]:
final_answer = sequential_chain("Italy", llm)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new SimpleSequentialChain chain...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What is the name of the most popular city in Italy for tourists?

The most popular city in Italy for tourists is Rome. Known for its ancient history, stunning architecture, and world-renowned landmarks such as the Colosseum and the Vatican, Rome attracts millions of visitors every year.
What are the top three things to do in this What is the name of the most popular city in Italy for tourists?

The most popular city in Italy for tourists is Rome. Known for its ancient history, stunning architecture, and world-renowned landmarks such as the Colosseum and the Vatican, Rome attracts millions of visitors every year. for tourists?

1. Visit the Colosseum: The Colosseum is one of the most iconic landmarks in Rome and a must-see for any visitor. This ancient amphitheater was built over 2,000 years ago and was used for gladiatorial contests, public spectacles, and dramas based on classical mythology.
2. Explore the Vatican: The Vatican is a sovereign city-state located within Rome and is the s

# Conclusions   


Mistral `7b-instruct-hf`, with careful adjusted queries, seems to work just fine.  
We had to adjust carefully the prompts to get a correct answer.